### Notebook to compile the facs data table from the FlowJo output (+IndexSort plugin).
.fcs files were gated in FlowJo and well location was preserved using the IndexSort plugin. Bi-exponential transform was applied and the FACS data was exported as the transformed 'channel' tables. To preserve the well location, also the un-transformed 'scale' tables were exported. These tables are beeing merged in this notebook.

In [5]:
import pandas as pd
import os

In [6]:
plates = ['8227_INX_celltype_P1_003',
          '8227_INX_celltype_P2_004',
          '8227_INX_celltype_P3_005']

path = '../data/facs_data/'

In [7]:
# get all csv files in channel and scale folder
files = [f for f in os.listdir(path+'channel/') if f.endswith(".csv")]
fcs = ['_'.join(x.split('_')[1:-1]) for x in files]
data = pd.DataFrame({'file': files, 'fcs': fcs, 'plate': [plates.index(p) for p in fcs]}).set_index('file')
dfs_channel = [pd.DataFrame() for i in range(len(plates))]

for f in files:
    fj = pd.read_csv(path+'channel/{}'.format(f))
    dfs_channel[data.loc[f, 'plate']] = dfs_channel[data.loc[f, 'plate']].append(fj)

dfs_scale = [pd.DataFrame() for i in range(len(plates))]
for f in files:
    fj = pd.read_csv(path+'scale/{}'.format(f))
    dfs_scale[data.loc[f, 'plate']] = dfs_scale[data.loc[f, 'plate']].append(fj)

# replace the index columns with the non-transformed values from scale
for i in range(len(dfs_channel)):
    dfs_channel[i].loc[:, ['IdxCol', 'IdxRow', 'Time']] = dfs_scale[i].loc[:, ['IdxCol', 'IdxRow', 'Time']]

# transform row index in letter and make Well column. Somehow, the IdxRow index from FJ is reversed
for i in range(len(dfs_channel)):
    dfs_channel[i]["IdxRow"] = dfs_channel[i]["IdxRow"].apply(
    lambda x: [
        "A",
        "B",
        "C",
        "D",
        "E",
        "F",
        "G",
        "H",
        "I",
        "J",
        "K",
        "L",
        "M",
        "N",
        "O",
        "P",
    ][-x]
    )
    dfs_channel[i]["Well"] = dfs_channel[i]["IdxRow"] + dfs_channel[i]["IdxCol"].astype(str)
    dfs_channel[i] = dfs_channel[i].rename(columns={'IdxRow': 'Row', 'IdxCol': 'Column'})

# save one table for each plate
[dfs_channel[i].to_csv(path+'facs_data_P{}.txt'.format(i+1), sep='\t', index=False) for i in range(len(dfs_channel))]

[None, None, None]